In [ ]:
%matplotlib inline
import holoviews as hv
hv.extension('bokeh', 'matplotlib',)
%opts Image[width=500, height=300, colorbar=True](cmap='magma') 
%opts QuadMesh[width=400,  height=200, colorbar=True](cmap='viridis')
%opts Curve[width=500, height=int(500/1.61)]



import xarray as xr
from os.path import join, abspath
import os
from scipy import quantile


RUN = abspath(os.getenv('RUN', '../models/17/test'))
TRAINING = "../data/processed/training.nc"


data_2d_path = join(RUN, 'OUT_2D', '*.nc')
data_3d_path = join(RUN, 'OUT_3D', '*.nc')
data_stat_path = join(RUN, 'OUT_STAT', '*.nc')
no_nn_path = '/Users/noah/workspace/research/uwnet/data/runs/2018-09-27/OUT_3D/*.nc'


print(data_2d_path)
data_2d = xr.open_mfdataset(data_2d_path)
data_3d = xr.open_mfdataset(data_3d_path)
stat = xr.open_mfdataset(data_stat_path)
train_data = xr.open_dataset(TRAINING)
no_nn = xr.open_mfdataset(no_nn_path)

data_3d['FQTNN'] *= 86400
data_3d['FQTNN'].attrs['units'] = 'g/kg/d'
data_3d['FSLINN'] *= 86400
data_3d['FSLINN'].attrs['units'] = 'K/d'

In [ ]:
data_2d.PW[::12].plot(col='time', col_wrap=3)

In [ ]:
data_2d.W500[::12].plot(col='time', col_wrap=3)

In [ ]:
data_3d.TABS

In [ ]:
data_3d.W[::12].mean('x').plot(col='time', col_wrap=3)

## Numerical Instability

The simulation diverged on day 9 (1 day before it could be completed). Why did this happen? Let's look at W in the last snapshot avaialable

In [ ]:
w = data_3d.W[-1]

In [ ]:
w[1].plot()

Here is the data at the previous time step:

In [ ]:
data_3d.W[-2, 1].plot()

There is a diverging vertical velocity in the second vertical level of the model.

In [ ]:
w[5].plot()

In [ ]:
w[15].plot()

This problem is not present at higher vertical levels

The temperature tendency predicted by the Neural network is not too large in this spot:

In [ ]:
data_3d.FSLINN[-1,1].plot()

# Forcing in a Single Location

In [ ]:
for loc in [dict(x=0, y=32), dict(x=0, y=10)]:
    for key in ['FQTNN', 'FSLINN', 'SLI', 'QT']:
        plt.figure()
        data_3d[key].isel(**loc).plot.contourf(x='time', levels=11)

# Zonal-Means

In [ ]:
data_3d['FQTNN']

In [ ]:
def get_plot_kwargs(x):
    kwargs = {}
    a,b = quantile(x, [0.02, .98])
    if a * b < 0:
        cmap = 'RdBu_r'
        m = max(abs(a), abs(b))
        a = -m 
        b = m
    else:
        cmap = 'viridis'
    return dict(levels=np.linspace(a, b, 11), cmap=cmap)

for key in ['FQTNN', 'FSLINN', 'SLI', 'QT']:
    plt.figure()
    x = data_3d[key][::12].mean('x')
    x.plot.contourf(col='time', **get_plot_kwargs(x))

# Error statistics

How do the two datasets compare in time.

In [ ]:
train_data.time

In [ ]:
data_3d.time

The training data is sampled twice as frequently. Let's align the two

In [ ]:
train_data_resampled = train_data.sel(time=data_3d.time, method='nearest')\
.assign_coords(x=data_3d.x, y=data_3d.y, time=data_3d.time)

no_nn_resampled = no_nn.sel(time=data_3d.time, method='nearest')\
.assign_coords(x=data_3d.x, y=data_3d.y, time=data_3d.time)

## Spatial structure of the error

In [ ]:
forecast_error = data_3d - train_data_resampled

forecast_error.QT[::12,1 ].plot(col='time', col_wrap=4)

In [ ]:
qt_error = train_data_resampled.QT -data_3d.QT

sli_error = train_data_resampled.SLI -data_3d.SLI
u_error = train_data_resampled.U -data_3d.U

u_error_persistence = train_data_resampled.U[0] - train_data_resampled.U

In [ ]:
sli_error[:,15].std('x').plot()

In [ ]:
qt_error[:,1].std('x').plot()

In [ ]:
u_error[:,15].std('x').plot()

## Comparison with persistence and null forecasts

In [ ]:
def rms(x):
    return np.sqrt((x**2).mean(['x', 'y']))

def error_plot(forecast, no_nn, truth):
    error  = rms(forecast- truth)
    persistence_error = rms(truth[0] - truth)
    time = error.time
    plt.plot(time,error, label='NN')
    plt.plot(time,persistence_error, label='Persistence Forecast')
    plt.plot(time,rms(no_nn-truth), label='No NN')
    plt.legend()
    
def plot_err_height_field(field, level):
    error_plot(data_3d[field][:,level], no_nn_resampled[field][:,level], train_data_resampled[field][:,level])
    plt.title(f"{field} RMS Error at z={float(data_3d.z[level])}")
    
def plot_err_field(field):
    levs = [5,10,15,20]

    for lev in levs:
        plt.figure()
        plot_err_height_field(field, lev)

In [ ]:
plot_err_field('SLI')

In [ ]:
plot_err_field('U')

In [ ]:
plot_err_field('V')

In [ ]:
plot_err_field('W')

In [ ]:
plot_err_field('QT')

In [ ]:
# !rm -rf /Users/noah/workspace/research/uwnet/data/runs/2018-09-27
# !python ../src/criticism/run_sam_ic.py -t 0 -o ../data/runs/ > out

# Error in P-E

In [ ]:
net_precip_nn = -(data_3d.FQTNN/1000 * train_data.layer_mass).sum('z')
net_precip_nn[::5].plot(col='time', col_wrap=5, vmax=50)

t_2d_plot= net_precip_nn[::5].time

In [ ]:
net_precip_obs = train_data.Prec - train_data.LHF/2.51e6 * 86400
net_precip_obs.sel(time=t_2d_plot, method='nearest').plot(col='time', col_wrap=5, vmax=50)

In [ ]:
net_precip_nn.mean('x')[-1].plot(label='NN+SAM')
net_precip_obs.mean(['x', 'time']).plot(label='OBS')
plt.title("Zonal mean net precipitation (mm/day)")

# After Chris meeting

Chris suggests:

1. Check for systematic bias in winds

In [ ]:
u_sfc_nn = data_3d.U.mean('x').isel(z=0)
u_sfc_obs = train_data_resampled.U.mean('x').isel(z=0)

In [ ]:

def zonal_mean_bias_plot(key, levels = (0, 5, 10 ,15, 20)):

    fig, axs = plt.subplots(len(levels), 2, figsize=(10,10), tight_layout=True, sharex=True, sharey=True)

    for k, lev in enumerate(levels):
        u_sfc_nn = data_3d[key].mean('x').isel(z=lev)
        u_sfc_obs = train_data_resampled[key].mean('x').isel(z=lev)

        M = max(u_sfc_obs.max(), u_sfc_nn.max()).compute()
        m = min(u_sfc_obs.min(), u_sfc_nn.min()).compute()
        M = max(abs(m), abs(M))

        u_sfc_obs.plot(ax=axs[k,0], vmax=M)
        u_sfc_nn.plot(ax=axs[k,1], vmax=M)
    axs[0,0].set_title('NGAqua')
    axs[0,1].set_title('SAM+NN')

In [ ]:
zonal_mean_bias_plot('U')

In [ ]:
zonal_mean_bias_plot('QT')

In [ ]:
zonal_mean_bias_plot('SLI')

In [ ]:
zonal_mean_bias_plot('W',levels=(1,5,10,15,20))

What is the behavior of the zonal mean net precipitation bias?

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,3))

kw = dict(vmax=20, vmin=-20, cmap='RdBu_r')

net_precip_nn.sel(time=net_precip_nn.time, method='nearest').mean('x').plot(label='NN+SAM', ax=axs[1], **kw)
net_precip_obs.sel(time=net_precip_nn.time, method='nearest').mean(['x']).plot(label='OBS', ax=axs[0], **kw)


plt.suptitle("Zonal Mean net precip (P-E) (mm/day)")
axs[0].set_title("NGAqua")
axs[1].set_title("SAM")

### Equatorial winds

Let's look at the difference in the zonal mean zonal velocity at the equator in NGAqua and in the truth.

In [ ]:
nn = data_3d.U.mean('x').isel(y=32)
obs = train_data_resampled.U.isel(y=32).mean('x')

In [ ]:
%%opts Curve{+framewise}
hv.Dataset(obs.load()).to.curve("time", label="SAM") \
   * hv.Dataset(nn.load()).to.curve("time", label="NN")

The surface easterlies in SAM/NN are much smaller than in NGAqua, but are larger in magnitude above 1km. As Chris suggests, this could indicate there is not enough vertical momentum mixing in the tropics of this model. This mixing could be increased by a good parametrization of CMT.